# Problem 1-1: Planting Optimization with Market Constraints

## Problem Statement

Optimize planting decisions for 2024-2030 where:
- **Production beyond expected sales is wasted** (no revenue, still incurs cost)
- Market capacity based on 2023 actual production (stable demand assumption)
- Crop classification from clustering analysis guides risk management

## Model Overview

**Two-stage MILP approach:**

1. **Stage 1:** Maximize total profit subject to all constraints
2. **Stage 2:** Minimize planting fragmentation while retaining ≥95% of Stage 1 profit

**Decision variables:**
- $a_{ijns}$: Planting area (mu) of crop $i$ on plot $j$ in year $n$, season $s$
- $H_{ijns} \in \{0,1\}$: Binary indicator for planting occurrence

In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import seaborn as sns

plt.rcParams['figure.dpi'] = 150
sns.set_theme(style='whitegrid')

## 1. Data Loading

In [2]:
# Load datasets
sales = pd.read_csv('sales_volume_data.csv')
sales = sales[sales['Crop Name'].notna()].copy()
farmland = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Existing Village Farmland')
crop_info = pd.read_excel('Attachment1_EN.xlsx', sheet_name='Village Crops')
planting_2023 = pd.read_excel('Attachment2_EN.xlsx', sheet_name='2023 Crop Planting Status')

# Clean data: strip whitespaces
farmland['Plot Type'] = farmland['Plot Type'].str.strip()
sales['Plot Type'] = sales['Plot Type'].str.strip()
sales['Crop Name'] = sales['Crop Name'].str.strip()

# Identify legume crops
bean_types = ['Grain (Legumes)', 'Vegetable (Legumes)']
bean_crops = set(crop_info[crop_info['Crop Type'].isin(bean_types)]['Crop Name'])

# Define scope
years = list(range(2024, 2031))
plots = farmland['Plot Name'].tolist()
crops = sales['Crop Name'].unique().tolist()

# Classify single-season plots
single_season_types = ['Flat Dry Land', 'Terraced Field', 'Hillside Land']
single_plots = farmland[farmland['Plot Type'].isin(single_season_types)]['Plot Name'].tolist()

print(f"Data loaded: {len(crops)} crops, {len(plots)} plots, {len(years)} years")

Data loaded: 41 crops, 54 plots, 7 years


## 2. Parameter Construction

### Plot Characteristics

$$A_j = \text{available land area for plot } j \text{ (mu)}$$

**Plot types:**
- Single-season: Flat Dry Land, Terraced Field, Hillside Land (1 season/year)
- Multi-season: Irrigated Land, Ordinary Greenhouse, Smart Greenhouse (2 seasons/year)

### Crop-Plot Economic Parameters

$$y_{ij} = \text{yield per mu of crop } i \text{ on plot type matching } j \text{ (jin/mu)}$$

$$c_{ij} = \text{production cost of crop } i \text{ on plot type matching } j \text{ (yuan/mu)}$$

$$p_i = \text{average selling price for crop } i \text{ (yuan/jin)}$$

### Market Capacity

$$Q_i = \text{annual market capacity for crop } i \text{ (jin)}$$

Based on 2023 actual production (stable demand assumption):

$$Q_i = \sum_{j,s} \text{Area}_{2023}^{ijs} \times y_{ij}$$

### Crop Classification

From clustering analysis (Section 3.1), crops are categorized into four risk-return classes:

| Class | Type | Characteristics |
|-------|------|----------------|
| 1 | Premium | High-risk, high-return (e.g., exotic mushrooms) |
| 2 | High-value | Facility-intensive (e.g., greenhouse vegetables) |
| 3 | Standard | Market-stable (e.g., common vegetables) |
| 4 | Basic | Low-risk grains (e.g., wheat, corn) |

In [3]:
# Extract plot characteristics
Area = {row['Plot Name']: row['Plot Area (Mu)'] for _, row in farmland.iterrows()}
PlotType = {row['Plot Name']: row['Plot Type'] for _, row in farmland.iterrows()}

# Build crop-plot parameters including crop class
Yield, Cost, Price, ExpectedSales, CropClass = {}, {}, {}, {}, {}

for _, row in sales.iterrows():
    crop = row['Crop Name']
    ptype = row['Plot Type']
    matching_plots = [p for p in plots if PlotType[p] == ptype]
    
    for p in matching_plots:
        Yield[(crop, p)] = row['Yield_per_mu']
        Cost[(crop, p)] = row['Cost_per_mu']
    
    Price[crop] = row['Avg_Price']
    ExpectedSales[crop] = row['Expected_Sales_Volume']
    CropClass[crop] = int(row['Class'])

# Check legume availability per plot
can_grow_beans = {plot: len([c for c in bean_crops if (c, plot) in Yield]) > 0 for plot in plots}

# Record 2023 legume history
beans_2023 = {}
for plot in plots:
    plot_data = planting_2023[planting_2023['Planting Plot'] == plot]
    has_beans = any(c in bean_crops for c in plot_data['Crop Name'].values)
    beans_2023[plot] = has_beans

# Season timeline for rotation constraints
seasons_timeline = [(y, s) for y in years for s in [1, 2]]

# Validation
print(f"Parameters built:")
print(f"  Crop-plot pairs: {len(Yield)}")
print(f"  Total market capacity: {sum(ExpectedSales.values()):,.0f} jin/year")
print(f"  Plots with legume capability: {sum(can_grow_beans.values())}")
print(f"  Plots with legumes in 2023: {sum(beans_2023.values())}")
print(f"\nCrop class distribution:")
for cls in [1, 2, 3, 4]:
    cls_crops = [c for c in crops if CropClass.get(c) == cls]
    print(f"  Class {cls}: {len(cls_crops)} crops")

Parameters built:
  Crop-plot pairs: 990
  Total market capacity: 2,833,300 jin/year
  Plots with legume capability: 54
  Plots with legumes in 2023: 19

Crop class distribution:
  Class 1: 2 crops
  Class 2: 3 crops
  Class 3: 35 crops
  Class 4: 1 crops


## 3. Constraint Function

### 3.1 Variable Linking (Big-M Method)

$$0.01 \cdot H_{ijns} \leq a_{ijns} \leq 10000 \cdot H_{ijns}$$

Ensures $a_{ijns} > 0$ only when $H_{ijns} = 1$, with minimum area 0.01 mu.

### 3.2 Rotation Constraint

$$H_{ijns} + H_{ijn(s+1)} \leq 1, \quad \forall i,j,n,s$$

No consecutive planting of the same crop (prevents soil degradation).

### 3.3 Land Capacity

**Single-season plots:**
$$\sum_{i} a_{ijn1} \leq A_j$$

**Multi-season plots:**
$$\sum_{i} a_{ijns} \leq A_j, \quad s \in \{1,2\}$$

### 3.4 Market Capacity

$$\sum_{j,s} a_{ijns} \cdot y_{ij} \leq Q_i, \quad \forall i,n$$

Annual production cannot exceed expected sales (excess is wasted in Problem 1-1).

### 3.5 Legume Rotation

**Annual limit:**
$$\sum_{i \in \mathcal{B}} \sum_{s} H_{ijns} \leq 1, \quad \forall j,n$$

**3-year minimum (accounting for 2023 history):**

If legumes planted in 2023:
$$\sum_{\tau=2024}^{2026} \sum_{s} \sum_{i \in \mathcal{B}} H_{ij\tau s} \geq 1$$

If no legumes in 2023:
$$\sum_{s} \sum_{i \in \mathcal{B}} (H_{ij,2024,s} + H_{ij,2025,s}) \geq 1$$

For subsequent windows:
$$\sum_{\tau=n}^{n+2} \sum_{s} \sum_{i \in \mathcal{B}} H_{ij\tau s} \geq 1, \quad n \in \{2024,...,2028\}$$

### 3.6 Risk Diversification (Optional)

**High-risk crop limit (Class 1):**
$$\sum_{i \in Class_1} \sum_{s} a_{ijns} \leq 0.30 \times \sum_j A_j, \quad \forall n$$

**Grain security (Class 4):**
$$\sum_{i \in Class_4} \sum_{s} a_{ijns} \geq 0.15 \times \sum_j A_j, \quad \forall n$$

In [4]:
def add_constraints(model, a, H, include_risk_constraints=False):
    """Add all constraints to the model."""
    M = 10000
    
    # (1) Variable linking
    for key in a:
        model.addConstr(a[key] <= M * H[key])
        model.addConstr(a[key] >= 0.01 * H[key])
    
    # (2) Rotation constraint
    for crop in crops:
        for plot in plots:
            for i in range(len(seasons_timeline) - 1):
                y1, s1 = seasons_timeline[i]
                y2, s2 = seasons_timeline[i + 1]
                k1, k2 = (crop, plot, y1, s1), (crop, plot, y2, s2)
                if k1 in H and k2 in H:
                    model.addConstr(H[k1] + H[k2] <= 1)
    
    # (3) Land capacity
    for plot in plots:
        for year in years:
            if plot in single_plots:
                model.addConstr(
                    gp.quicksum(a[k] for k in a if k[1]==plot and k[2]==year and k[3]==1) <= Area[plot]
                )
            else:
                for s in [1, 2]:
                    model.addConstr(
                        gp.quicksum(a[k] for k in a if k[1]==plot and k[2]==year and k[3]==s) <= Area[plot]
                    )
    
    # (4) Market capacity (annual)
    for crop in crops:
        for year in years:
            model.addConstr(
                gp.quicksum(a[k] * Yield[(k[0],k[1])] for k in a if k[0]==crop and k[2]==year) <= ExpectedSales[crop]
            )
    
    # (5) Legume rotation
    for plot in plots:
        if not can_grow_beans[plot]:
            continue
        
        # At most 1 legume per year
        for year in years:
            model.addConstr(
                gp.quicksum(H[k] for k in H if k[0] in bean_crops and k[1]==plot and k[2]==year) <= 1
            )
        
        # 3-year minimum
        if not beans_2023.get(plot, False):
            model.addConstr(
                gp.quicksum(H[k] for k in H if k[0] in bean_crops and k[1]==plot and k[2] in [2024,2025]) >= 1
            )
        
        for i in range(1, len(years) - 2):
            window = years[i:i + 3]
            model.addConstr(
                gp.quicksum(H[k] for k in H if k[0] in bean_crops and k[1]==plot and k[2] in window) >= 1
            )
    
    # (6) Optional: Risk diversification
    if include_risk_constraints:
        total_area = sum(Area.values())
        class1_crops = [c for c in crops if CropClass.get(c) == 1]
        class4_crops = [c for c in crops if CropClass.get(c) == 4]
        
        if len(class1_crops) > 0:
            for year in years:
                model.addConstr(
                    gp.quicksum(a[k] for k in a if k[0] in class1_crops and k[2]==year) <= 0.30 * total_area
                )
        
        if len(class4_crops) > 0:
            for year in years:
                model.addConstr(
                    gp.quicksum(a[k] for k in a if k[0] in class4_crops and k[2]==year) >= 0.15 * total_area
                )

## 4. Stage 1: Maximize Profit

### Objective Function

$$\max Z_1 = \sum_{i,j,n,s} a_{ijns} \cdot (p_i \cdot y_{ij} - c_{ij})$$

where:
- $p_i \cdot y_{ij}$: revenue per mu
- $c_{ij}$: cost per mu
- $p_i \cdot y_{ij} - c_{ij}$: net profit per mu

Total profit aggregated over all crops, plots, years (2024-2030), and seasons.

In [5]:
model_s1 = gp.Model("Stage1")
model_s1.setParam('OutputFlag', 0)
model_s1.setParam('MIPGap', 0.01)

# Create variables
a, H = {}, {}
for crop in crops:
    for plot in plots:
        if (crop, plot) not in Yield:
            continue
        for year in years:
            seasons = [1] if plot in single_plots else [1, 2]
            for s in seasons:
                key = (crop, plot, year, s)
                a[key] = model_s1.addVar(lb=0, ub=Area[plot], name=f"a_{crop}_{plot}_{year}_S{s}")
                H[key] = model_s1.addVar(vtype=GRB.BINARY, name=f"H_{crop}_{plot}_{year}_S{s}")

model_s1.update()

# Objective function
profit = gp.quicksum(a[k] * (Price[k[0]] * Yield[(k[0],k[1])] - Cost[(k[0],k[1])]) for k in a)
model_s1.setObjective(profit, GRB.MAXIMIZE)

add_constraints(model_s1, a, H, include_risk_constraints=False)

model_s1.optimize()
P_star = model_s1.ObjVal

print(f"Stage 1 optimal profit: ¥{P_star:,.0f}")
print(f"Average per year: ¥{P_star/7:,.0f}")

Set parameter Username
Set parameter LicenseID to value 2735705
Academic license - for non-commercial use only - expires 2026-11-10
Stage 1 optimal profit: ¥60,483,539
Average per year: ¥8,640,506


## 5. Stage 2: Minimize Fragmentation

### Objective Function

$$\min Z_2 = \sum_{i,j,n,s} H_{ijns}$$

Minimizes the total number of planting decisions (crop-plot-season combinations).

### Profit Constraint

$$Z_1' = \sum_{i,j,n,s} a_{ijns} \cdot (p_i \cdot y_{ij} - c_{ij}) \geq 0.95 \times Z_1^*$$

where $Z_1^*$ is the optimal profit from Stage 1.

Ensures final profit maintains at least 95% of theoretical maximum.

In [6]:
model_s2 = model_s1.copy()
model_s2.setParam('OutputFlag', 0)
model_s2.setParam('MIPGap', 0.06)

# Retrieve variables from copied model
a2 = {k: model_s2.getVarByName(f"a_{k[0]}_{k[1]}_{k[2]}_S{k[3]}") for k in a}
H2 = {k: model_s2.getVarByName(f"H_{k[0]}_{k[1]}_{k[2]}_S{k[3]}") for k in H}

# New objective: minimize fragmentation
model_s2.setObjective(gp.quicksum(H2.values()), GRB.MINIMIZE)

# Profit constraint: retain ≥95% of Stage 1 profit
profit2 = gp.quicksum(a2[k] * (Price[k[0]] * Yield[(k[0],k[1])] - Cost[(k[0],k[1])]) for k in a2)
model_s2.addConstr(profit2 >=  0.95 * P_star)

model_s2.optimize()
final_profit = profit2.getValue()


print(f"Stage 2 final profit: ¥{final_profit:,.0f} ({final_profit/P_star*100:.1f}%)")
print(f"Average per year: ¥{final_profit/7:,.0f}")

Stage 2 final profit: ¥57,459,362 (95.0%)
Average per year: ¥8,208,480


## 6. Extract Results

In [12]:
# Extract all non-zero planting decisions
results = []
for k in a2:
    if a2[k].X > 0.01:
        crop, plot, year, season = k
        results.append({
            'Year': year,
            'Plot': plot,
            'Season': season,
            'Crop': crop,
            'Area': a2[k].X,
            'Class': CropClass.get(crop, 0)
        })

df_results = pd.DataFrame(results)
df_results = df_results.sort_values(['Year', 'Plot', 'Season'])

print(f"Extracted {len(results)} planting decisions")

Extracted 443 planting decisions


## 7. Export to Excel Template

In [19]:
import openpyxl

wb = openpyxl.load_workbook('result1_1_EN.xlsx')

# Get crop columns
ws_sample = wb['2024']
crop_to_col = {}
for col_idx in range(3, ws_sample.max_column + 1):
    crop_name = ws_sample.cell(1, col_idx).value
    if crop_name:
        crop_to_col[crop_name] = col_idx

# Build plot-row mapping
def get_plot_rows(ws):
    season1, season2 = {}, {}
    current_season = None
    for row_idx in range(2, 85):  # Stop before note rows (row 85+)
        marker = ws.cell(row_idx, 1).value
        plot = ws.cell(row_idx, 2).value
        if marker == "Season 1":
            current_season = 1
        elif marker == "Season 2":
            current_season = 2
        if plot and current_season == 1:
            season1[plot] = row_idx
        elif plot and current_season == 2:
            season2[plot] = row_idx
    return season1, season2

# Fill each year
for year in years:
    ws = wb[str(year)]
    season1_map, season2_map = get_plot_rows(ws)
    valid_rows = set(season1_map.values()) | set(season2_map.values())
    
    for row_idx in valid_rows:
        for col_idx in range(3, ws.max_column + 1):
            ws.cell(row_idx, col_idx).value = 0
    
    year_data = df_results[df_results['Year'] == year]
    for _, row in year_data.iterrows():
        plot, season, crop, area = row['Plot'], int(row['Season']), row['Crop'], round(row['Area'], 2)
        row_idx = season1_map.get(plot) if season == 1 else season2_map.get(plot)
        col_idx = crop_to_col.get(crop)
        if row_idx and col_idx:
            ws.cell(row_idx, col_idx).value = area

wb.save('result1_1_OUTPUT.xlsx')
print("✓ Saved to result1_1_OUTPUT.xlsx")

✓ Saved to result1_1_OUTPUT.xlsx


## 11. Validation

In [13]:
print("\n" + "="*60)
print("VALIDATION CHECKS")
print("="*60)

# Check rotation violations
rotation_violations = 0
for crop in crops:
    for plot in plots:
        for i in range(len(seasons_timeline) - 1):
            y1, s1 = seasons_timeline[i]
            y2, s2 = seasons_timeline[i + 1]
            k1, k2 = (crop, plot, y1, s1), (crop, plot, y2, s2)
            if k1 in a2 and k2 in a2:
                if a2[k1].X > 0.01 and a2[k2].X > 0.01:
                    rotation_violations += 1

print(f"Rotation violations: {rotation_violations}")

# Check market capacity
market_violations = 0
for crop in crops:
    for year in years:
        total_prod = sum(a2[k].X * Yield[(k[0], k[1])] for k in a2 if k[0] == crop and k[2] == year)
        if total_prod > ExpectedSales[crop] + 1:
            market_violations += 1
            print(f"  Warning: {crop} in {year} exceeds capacity by {total_prod - ExpectedSales[crop]:.0f} jin")

print(f"Market capacity violations: {market_violations}")

# Summary
if rotation_violations == 0 and market_violations == 0:
    print("\n✓ All constraints satisfied")
else:
    print(f"\n⚠ Found {rotation_violations + market_violations} constraint violations")

print("="*60)


VALIDATION CHECKS
Rotation violations: 0
Market capacity violations: 0

✓ All constraints satisfied
